In [341]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import SGDClassifier
from gensim.models import word2vec
from sklearn.ensemble import RandomForestClassifier
from pymystem3 import Mystem
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import gensim.sklearn_api
from xgboost import XGBClassifier
from gensim.sklearn_api import W2VTransformer
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from emoji import UNICODE_EMOJI
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from string import punctuation
from keras import models
from keras import layers
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score
import warnings
import time
warnings.filterwarnings('ignore')
%matplotlib inline

# Загрузка датасета, разделение на текстовые данные и таргет

In [351]:
df = pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


Судя по датасету, работать предстоит только с признаком "text" и таргетом "airline_sentiment". Остальные признаки выступают скорее для всевозможных исследовательских работ (определить, как распределены положительные отзывы среди авиакомпаний и т.п.)

In [352]:
texts = pd.DataFrame(df.text, columns=['text'])
Y = pd.DataFrame(df.airline_sentiment, columns=['airline_sentiment'])

In [353]:
for i in Y.airline_sentiment.unique():
    print('Доля класса {}: {:.2%}'.format(i, len(Y[Y['airline_sentiment']==i]) / len(Y)))

Доля класса neutral: 21.17%
Доля класса positive: 16.14%
Доля класса negative: 62.69%


Классы имеют выраженный дисбаланс.

Переобозначим классы на -1, 0, 1

In [354]:
d = {'negative':-1, 'neutral':0, 'positive':1}
Y['airline_sentiment'] = Y['airline_sentiment'].map(d)

Для валидации отложим 10% данных

In [355]:
X, hold_out_X, y, hold_out_y = train_test_split(texts, Y, test_size=0.1)

Используем RandomOverSampler для балансировки классов в датасете X

In [356]:
ros = RandomOverSampler()
X, y = ros.fit_resample(X, y)
print('Распределение классов после RandomOverSampler: {}'.format(sorted(Counter(y).items())))
X = pd.DataFrame(X, columns=['text'])
y = pd.DataFrame(y, columns=['airline_sentiment'])

Распределение классов после RandomOverSampler: [(-1, 8279), (0, 8279), (1, 8279)]


In [226]:
tweets = [t for t in X.text]
hold_out_tweets = [t for t in hold_out_X.text]

# Препроцессинг текста твитов

Просмотрев твиты, обнаружил, что практически все записи имеют хештеги, ряд записей имеют ссылки, пунктуацию (в том числе искаженную (!!!! и т.п.), эмодзи). 

Две дополнительные функции для обработки текста. Первая соединяет слова no и not со следующим словом через нижнее подчеркивание (чтобы избегать ситуаций, когда эти слова неправильно интерпретируются: not goog/ not bad -> not_good/not_bad).
Вторая функция проверяет, является ли токен эмодзи. Во многих случаях эмодзи помогают определить эмоциональный окрас текста/твита.

In [227]:
def no_preprocessing(texts):
    no_prepr = []
    for text in texts:
        
        tmp = ''
        
        for i in text.split():
            if i.lower() == 'no':
                tmp += 'no'
            elif i.lower() == 'not':
                tmp += 'not'
            else:
                tmp += i + ' '
        no_prepr.append(tmp)
    
    return no_prepr

def is_emoji(s):
    return s in UNICODE_EMOJI

Первый препроцессинг: (no_preprocessing+len(token)>3)

In [228]:
def prep_1(texts):
    
    mystem = Mystem() 
    sw = stopwords.words("english")
        
    extend_list = ['site', 'http', 'https', 'rt', 'rt:']
    sw.extend(extend_list)

    sw.pop(sw.index('no'))
    sw.pop(sw.index('not'))
    
    clear_tweets = []
    
    for text in texts:
        text = re.sub('\@(\w+)', '', text) # удаление названий авиакомпаний
        text = re.sub('#(\w+)', " ", text) # удаление хештегов
        text = re.sub("[^a-zA-Z,_]", " ", text) # удаление лишних спецсимволов
        text = text.strip(" ")
        
        tokens = mystem.lemmatize(text.lower())
        tokens = [token for token in tokens if token not in sw 
                  and token != " " 
                  and token.strip() not in punctuation 
                  and len(token)>1]
        
        text = " ".join(tokens)
        
        clear_tweets.append(text)
        
    return no_preprocessing(clear_tweets)

Второй препроцессинг: (len(token)>2)

In [229]:
def prep_2(texts):
    
    mystem = Mystem() 
    sw = stopwords.words("english")
        
    extend_list = ['site', 'http', 'https', 'rt', 'rt:']
    sw.extend(extend_list)

    clear_tweets = []
    
    for text in texts:
        text = re.sub('\@(\w+)', '', text) # удаление названий авиакомпаний
        text = re.sub('#(\w+)', " ", text) # удаление хештегов
        text = re.sub("[^a-zA-Z,_]", " ", text) # удаление лишних спецсимволов
        text = text.strip(" ")
        
        tokens = mystem.lemmatize(text.lower())
        tokens = [token for token in tokens if token not in sw 
                  and token != " " 
                  and token.strip() not in punctuation 
                  and len(token)>2]
        
        text = " ".join(tokens)
        
        clear_tweets.append(text)
        
    return clear_tweets

Третий препроцессинг: (no_preprocessing+is_emoji+len(token)>2)

In [230]:
def prep_3(texts):
    
    mystem = Mystem() 
    sw = stopwords.words("english")
        
    extend_list = ['site', 'http', 'https', 'rt', 'rt:']
    sw.extend(extend_list)

    sw.pop(sw.index('no'))
    sw.pop(sw.index('not'))
    
    clear_tweets = []
    
    for text in texts:
        text = re.sub('\@(\w+)', '', text) # удаление названий авиакомпаний
        text = re.sub('#(\w+)', " ", text) # удаление хештегов
        text = re.sub('[#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n"1234567890]', " ", text) #удаление шума
        text = text.strip(" ")
        
        tokens = mystem.lemmatize(text.lower())
        tokens = [token for token in tokens if token not in sw 
                  and token != " " 
                  and token.strip() not in punctuation 
                  and (len(token)>2 or is_emoji(token))]
        
        text = " ".join(tokens)
        
        clear_tweets.append(text)
        
    return no_preprocessing(clear_tweets)

Четвертый препроцессинг: (is_emoji+len(token)>2)

In [231]:
def prep_4(texts):
    
    mystem = Mystem() 
    sw = stopwords.words("english")
        
    extend_list = ['site', 'http', 'https', 'rt', 'rt:']
    sw.extend(extend_list)
    
    clear_tweets = []
    
    for text in texts:
        text = re.sub('\@(\w+)', '', text) # удаление названий авиакомпаний
        text = re.sub('#(\w+)', " ", text) # удаление хештегов
        text = re.sub('[#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n"1234567890]', " ", text) #удаление шума
        text = text.strip(" ")
        
        tokens = mystem.lemmatize(text.lower())
        tokens = [token for token in tokens if token not in sw 
                  and token != " " 
                  and token.strip() not in punctuation 
                  and (len(token)>2 or is_emoji(token))]
        
        text = " ".join(tokens)
        
        clear_tweets.append(text)
        
    return clear_tweets

Данные с учетом препроцессинга

In [232]:
preprocess_tweets_for_train = [('no_prepr', tweets), ('prepr_train_1', prep_1(tweets)), 
                               ('prepr_train_2', prep_2(tweets)), ('prepr_train_3', prep_3(tweets)), 
                               ('prepr_train_4', prep_4(tweets))]

preprocess_hold_out_tweets = [('no_prepr', hold_out_tweets), ('prep_hold_out_1', prep_1(hold_out_tweets)), ('prep_hold_out_2', prep_2(hold_out_tweets)),
                               ('prep_hold_out_3', prep_3(hold_out_tweets)), ('prep_hold_out_4', prep_4(hold_out_tweets))]

# Преобразование признаков и обучение

## CountVectorizer

Учитывая, что твиты короткие, будем использовать n-граммы (биграммы/триграммы). Посмотрим на качество работы классификаторов на кросс-валидации, а также на отложенной выборке.

In [233]:
c_vectorizer = CountVectorizer(ngram_range=(1,3))

classifiers = [('LogisticRegression', LogisticRegression(class_weight='balanced')), 
              ('SGDClassifier', SGDClassifier(class_weight='balanced')), 
              ('LinearSVC', LinearSVC(class_weight='balanced')), 
              ('XGBClassifier', XGBClassifier(n_estimators=100, learning_rate=0.5, n_jobs=-1)),
              ('RandomForestClassifier', RandomForestClassifier(class_weight='balanced', n_estimators=100)),
              ('MultinomialNB', MultinomialNB()),
              ('BernoulliNB', BernoulliNB()),
              ('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=3))]

In [234]:
scores_cvs_cv = []
scores_ho = []

for c in classifiers:
    
    pipeline = Pipeline([('Count_vectorizer', c_vectorizer), c])
    
    for pt in preprocess_tweets_for_train:
        cvs = cross_val_score(pipeline, pt[1], y, cv=4)
        scores_cvs_cv.append((c[0], pt[0], cvs.mean()))
        pipeline.fit(pt[1], y)
        
    for ph in preprocess_hold_out_tweets:
        pred = pipeline.predict(ph[1])
        acc_holdout = accuracy_score(hold_out_y, pred)
        scores_ho.append((c[0], ph[0], acc_holdout))

In [235]:
scores_cvs_cv = sorted(scores_cvs_cv, key=lambda x: x[2], reverse=True)
scores_ho = sorted(scores_ho, key=lambda x: x[2], reverse=True)

In [236]:
scores_cvs_cv[:10]

[('SGDClassifier', 'no_prepr', 0.9493126167205883),
 ('RandomForestClassifier', 'no_prepr', 0.9468085647704542),
 ('MultinomialNB', 'no_prepr', 0.9436176344690067),
 ('LinearSVC', 'no_prepr', 0.9409112002915901),
 ('LogisticRegression', 'no_prepr', 0.9407899392645868),
 ('SGDClassifier', 'prepr_train_4', 0.9310159990969223),
 ('SGDClassifier', 'prepr_train_1', 0.9301271886480265),
 ('SGDClassifier', 'prepr_train_3', 0.929238769616261),
 ('SGDClassifier', 'prepr_train_2', 0.9281882256090764),
 ('LogisticRegression', 'prepr_train_1', 0.9190605738363012)]

In [237]:
scores_ho[:10]

[('SGDClassifier', 'no_prepr', 0.7903005464480874),
 ('SGDClassifier', 'prep_hold_out_2', 0.7868852459016393),
 ('SGDClassifier', 'prep_hold_out_4', 0.7868852459016393),
 ('SGDClassifier', 'prep_hold_out_3', 0.782103825136612),
 ('SGDClassifier', 'prep_hold_out_1', 0.7786885245901639),
 ('MultinomialNB', 'prep_hold_out_2', 0.7780054644808743),
 ('MultinomialNB', 'prep_hold_out_3', 0.7780054644808743),
 ('MultinomialNB', 'prep_hold_out_4', 0.7780054644808743),
 ('LogisticRegression', 'prep_hold_out_2', 0.7773224043715847),
 ('LogisticRegression', 'prep_hold_out_4', 0.7773224043715847)]

Исходя из результатов можно сделать следующие выводы:

1. На трейне имеем очень высокую точность по сравнению с отложенной выборкой = переобучение
2. Препроцессинг в некоторых классификаторах оказывает влияние на качество его работы, однако практически везде разница незначительна.
3. Для дальнейшего поиска параметров по сетке будем использовать классификаторы, показавшие лучшее качество на отложенной выборке = LogisticRegression и SGDClassifier

In [238]:
pipeline_cv_lr = Pipeline([('CountVectorizer', CountVectorizer()), 
                            ('clf', LogisticRegression(class_weight='balanced'))])

parameters = {
    'CountVectorizer__max_df': (0.5, 0.75, 1.0),
    'CountVectorizer__ngram_range': ((1, 1), (1, 2), (1, 3)),
    'clf__tol': [0.001, 0.0001],
    'clf__penalty': ['l2', 'l1']}

grid_search = GridSearchCV(pipeline_cv_lr, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(preprocess_tweets_for_train[0][1], y)
best_parameters = grid_search.best_estimator_.get_params()
print("Best score: %0.5f" % grid_search.best_score_)
print("Best parameters set for SGDClassifier:")
print('max_df =', best_parameters['CountVectorizer__max_df'])
print('ngram_range =', best_parameters['CountVectorizer__ngram_range'])
print('alpha =', best_parameters['clf__tol'])
print('penalty =', best_parameters['clf__penalty'])

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  2.2min finished


Best score: 0.94079
Best parameters set for SGDClassifier:
max_df = 0.5
ngram_range = (1, 3)
alpha = 0.001
penalty = l2


In [239]:
pipeline_cv_lr = Pipeline([('CountVectorizer', CountVectorizer(max_df=0.5, ngram_range=(1, 3))), 
                            ('clf', LogisticRegression(class_weight='balanced'))])
pipeline_cv_lr.fit(tweets, y)
accuracy_score(hold_out_y, pipeline_cv_lr.predict(hold_out_tweets))

0.8121584699453552

In [240]:
pipeline_cv_sgd = Pipeline([('CountVectorizer', CountVectorizer()), 
                            ('clf', SGDClassifier(class_weight='balanced'))])

parameters = {
    'CountVectorizer__max_df': (0.5, 0.75, 1.0),
    'CountVectorizer__ngram_range': ((1, 1), (1, 2), (1, 3)),
    'clf__alpha': [0.001, 0.0001, 0.00001],
    'clf__penalty': [None, 'l2', 'l1']}

grid_search = GridSearchCV(pipeline_cv_sgd, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(preprocess_tweets_for_train[0][1], y)
best_parameters = grid_search.best_estimator_.get_params()
print("Best score: %0.5f" % grid_search.best_score_)
print("Best parameters set for SGDClassifier:")
print('max_df =', best_parameters['CountVectorizer__max_df'])
print('ngram_range =', best_parameters['CountVectorizer__ngram_range'])
print('alpha =', best_parameters['clf__alpha'])
print('penalty =', best_parameters['clf__penalty'])

Fitting 4 folds for each of 81 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:  3.1min finished


Best score: 0.95210
Best parameters set for SGDClassifier:
max_df = 0.5
ngram_range = (1, 3)
alpha = 1e-05
penalty = None


In [241]:
pipeline_cv_sgd = Pipeline([('CountVectorizer', CountVectorizer(max_df=0.5, ngram_range=(1,3))), 
                     ('clf', SGDClassifier(class_weight='balanced'))])
pipeline_cv_sgd.fit(tweets, y)
accuracy_score(hold_out_y, pipeline_cv_sgd.predict(hold_out_tweets))

0.8121584699453552

В результате получили два пайплайна с точностью около 0.8

## Tfidif vectorizer

Повторим на тех же классификаторах

In [242]:
t_vectorizer = TfidfVectorizer(ngram_range=(1,3))

In [243]:
scores_cvs_tv = []
scores_ho_tv = []

for c in classifiers:
    
    pipeline = Pipeline([('Tfidf_vectorizer', t_vectorizer), c])
    
    for pt in preprocess_tweets_for_train:
        cvs = cross_val_score(pipeline, pt[1], y, cv=4)
        scores_cvs_tv.append((c[0], pt[0], cvs.mean()))
        pipeline.fit(pt[1], y)
        
    for ph in preprocess_hold_out_tweets:
        pred = pipeline.predict(ph[1])
        acc_holdout = accuracy_score(hold_out_y, pred)
        scores_ho_tv.append((c[0], ph[0], acc_holdout))

In [244]:
scores_cvs_tv = sorted(scores_cvs_tv, key=lambda x: x[2], reverse=True)
scores_ho_tv = sorted(scores_ho_tv, key=lambda x: x[2], reverse=True)

In [245]:
scores_cvs_tv[:5]

[('LinearSVC', 'no_prepr', 0.9504836584913718),
 ('LinearSVC', 'prepr_train_4', 0.9367110204902171),
 ('LinearSVC', 'prepr_train_2', 0.936670626242358),
 ('LinearSVC', 'prepr_train_3', 0.9359840218830359),
 ('LinearSVC', 'prepr_train_1', 0.9356205127940169)]

In [246]:
scores_ho_tv[:10]

[('SGDClassifier', 'prep_hold_out_2', 0.7896174863387978),
 ('SGDClassifier', 'prep_hold_out_4', 0.7896174863387978),
 ('SGDClassifier', 'prep_hold_out_3', 0.787568306010929),
 ('SGDClassifier', 'prep_hold_out_1', 0.7868852459016393),
 ('LinearSVC', 'prep_hold_out_2', 0.7862021857923497),
 ('LinearSVC', 'prep_hold_out_4', 0.7862021857923497),
 ('LogisticRegression', 'prep_hold_out_2', 0.7814207650273224),
 ('LogisticRegression', 'prep_hold_out_4', 0.7814207650273224),
 ('LinearSVC', 'prep_hold_out_3', 0.7814207650273224),
 ('LogisticRegression', 'prep_hold_out_1', 0.7800546448087432)]

In [247]:
pipeline = Pipeline([('TfidfVectorizer', TfidfVectorizer()), ('clf', LinearSVC(class_weight='balanced'))])

parameters = {
    'TfidfVectorizer__max_df': (0.5, 0.25, 1.0),
    'TfidfVectorizer__ngram_range': ((1, 1), (1, 2), (1, 3)),
    'clf__tol': [0.001, 0.0001, 0.00001],
    'clf__C': [0.1, 1, 10]}

grid_search = GridSearchCV(pipeline, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(preprocess_tweets_for_train[0][1], y)
best_parameters = grid_search.best_estimator_.get_params()
print("Best score: %0.5f" % grid_search.best_score_)
print("Best parameters set for LinearSVC:")
print('max_df =', best_parameters['TfidfVectorizer__max_df'])
print('ngram_range =', best_parameters['TfidfVectorizer__ngram_range'])
print('tol =', best_parameters['clf__tol'])
print('C =', best_parameters['clf__C'])

Fitting 4 folds for each of 81 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:  4.7min finished


Best score: 0.95335
Best parameters set for LinearSVC:
max_df = 0.5
ngram_range = (1, 3)
tol = 0.001
C = 10


In [248]:
pipeline_tv_lsvc = Pipeline([('CountVectorizer', CountVectorizer(max_df=0.5, ngram_range=(1,3))), 
                     ('clf', LinearSVC(class_weight='balanced', tol=0.001, C=10))])
pipeline_tv_lsvc.fit(tweets, y)
accuracy_score(hold_out_y, pipeline_tv_lsvc.predict(hold_out_tweets))

0.8046448087431693

In [249]:
pipeline = Pipeline([('TfidfVectorizer', TfidfVectorizer()), ('clf', SGDClassifier(class_weight='balanced'))])

parameters = {
    'TfidfVectorizer__max_df': (0.25, 0.5, 0.75, 1.0),
    'TfidfVectorizer__ngram_range': ((1, 1), (1, 2), (1, 3)),
    'clf__alpha': [0.001, 0.0001, 0.00001],
    'clf__penalty': ['l2', 'l1']}

grid_search = GridSearchCV(pipeline, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(preprocess_tweets_for_train[0][1], y)
best_parameters = grid_search.best_estimator_.get_params()
print("Best score: %0.5f" % grid_search.best_score_)
print("Best parameters set for LinearSVC:")
print('max_df =', best_parameters['TfidfVectorizer__max_df'])
print('ngram_range =', best_parameters['TfidfVectorizer__ngram_range'])
print('alpha =', best_parameters['clf__alpha'])
print('penalty =', best_parameters['clf__penalty'])

Fitting 4 folds for each of 72 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:  2.2min finished


Best score: 0.95319
Best parameters set for LinearSVC:
max_df = 0.75
ngram_range = (1, 3)
alpha = 1e-05
penalty = l2


In [250]:
pipeline_tv_sgd = Pipeline([('TfidfVectorizer', TfidfVectorizer(ngram_range=(1,3), max_df=1.0)), 
                     ('clf', SGDClassifier(class_weight='balanced', alpha=1e-05))])
pipeline_tv_sgd.fit(tweets, y)
accuracy_score(hold_out_y, pipeline_tv_sgd.predict(hold_out_tweets))

0.8080601092896175

Пайплайны с TfidfVectorizer отработали сравнимо по качеству с пайплайнами с CountVectorizer. Пока лучшее качество у TfidfVectorizer+SGDClassifier (0.812).

## Word2vec

Построим словарь word2vec и создадим класс с функциями fit и transform

In [251]:
ct_list = [i.split() for i in preprocess_tweets_for_train[0][1]]
model = word2vec.Word2Vec(min_count=3)
model.build_vocab(ct_list)
model.train(ct_list, total_examples=model.corpus_count, epochs=model.iter)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

Создадим отдельный класс с функциями fit, transform для w2v

In [252]:
class MeanVect(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(10)], axis=0) for words in X])

Будем использовать следующие классификаторы:

In [253]:
classifiers2 = [('LogisticRegression', LogisticRegression(class_weight='balanced')), 
              ('SGDClassifier', SGDClassifier(class_weight='balanced')), 
              ('LinearSVC', LinearSVC(class_weight='balanced')), 
              ('XGBClassifier', XGBClassifier(n_estimators=100, learning_rate=0.5, n_jobs=-1)),
              ('RandomForestClassifier', RandomForestClassifier(class_weight='balanced', n_estimators=100, n_jobs=-1)),
              ('BernoulliNB', BernoulliNB())]

Оценим качество на кросс-валидации и на отложенной выборке

In [254]:
for c in classifiers2:
    
    pipeline = Pipeline([("word2vec", MeanVect(w2v)), c])
    
    cvs = cross_val_score(pipeline, preprocess_tweets_for_train[0][1], y, cv=4)
    print('cvs:', c[0], cvs.mean())
    pipeline.fit(preprocess_tweets_for_train[0][1], y)
        
    pred = pipeline.predict(hold_out_tweets)
    acc_holdout = accuracy_score(hold_out_y, pred)
    print('ho:', acc_holdout)

cvs: LogisticRegression 0.50785570266577
ho: 0.5218579234972678
cvs: SGDClassifier 0.46023276324211126
ho: 0.3025956284153005
cvs: LinearSVC 0.5421863316198657
ho: 0.5594262295081968
cvs: XGBClassifier 0.7005540548622775
ho: 0.5689890710382514
cvs: RandomForestClassifier 0.9211618184175225
ho: 0.6468579234972678
cvs: BernoulliNB 0.42089761185449054
ho: 0.5594262295081968


Качество на моделях с Word2vec оказалось значительно ниже, чем на пайплайнах ранее - самый высокий показатель accuracy на пайплайне word2vec+RandomForestClassifier (0.636), поэтому дополнительные процедуры (поиск по сетке, прогон на других данных (другие препроцессинги) и тд) не целесообразны (по крайней мере в той постановке задачи, которая решается)

## Vowpal wabbit

Обозначим классы так, чтобы не было отрицательных

In [255]:
all_documents = X.text
topic_encoder = LabelEncoder()
all_targets_mult = topic_encoder.fit_transform(y.airline_sentiment) + 1

Разделим на train/test

In [256]:
train_documents, test_documents, train_labels_mult, test_labels_mult = \
    train_test_split(all_documents, all_targets_mult)

Функция для подгонки данных к формату vw

In [257]:
def to_vw_format(document, label=None):
    return str(label or '') + ' |text ' + ' '.join(re.findall('\w{3,}', document.lower())) + '\n'

In [258]:
with open('train.vw', 'w') as vw_train_data:
    for text, target in zip(train_documents, train_labels_mult):
        vw_train_data.write(to_vw_format(text, target))
with open('test.vw', 'w') as vw_test_data:
    for text in test_documents:
        vw_test_data.write(to_vw_format(text))

One against all на 3 класса с функцией потерь hinge:

In [259]:
%%time
!vw --oaa 3 train.vw -f model.vw --loss_function=hinge

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        3        1        6
1.000000 1.000000            2            2.0        1        3        7
0.750000 0.500000            4            4.0        1        1       19
0.750000 0.750000            8            8.0        3        1       10
0.687500 0.625000           16           16.0        2        3        9
0.656250 0.625000           32           32.0        3        3        4
0.625000 0.593750           64           64.0        1        3       17
0.539062 0.453125          128          128.0        1        1       14
0.496094 0.453125          256          256.0        1        3       18
0.423828 0.351562          512          51

In [260]:
%%time
!vw -i model.vw -t -d test.vw -p predictions.txt

only testing
predictions = predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
    n.a.     n.a.            1            1.0  unknown        3       10
    n.a.     n.a.            2            2.0  unknown        1        9
    n.a.     n.a.            4            4.0  unknown        3        5
    n.a.     n.a.            8            8.0  unknown        1       23
    n.a.     n.a.           16           16.0  unknown        3       19
    n.a.     n.a.           32           32.0  unknown        2       12
    n.a.     n.a.           64           64.0  unknown        1       19
    n.a.     n.a.          128          128.0  unknown        1       21
    n.a.     n.a.          256          256.0  unknown        2       25
    n.a.     n.a.          

In [261]:
with open('predictions.txt') as pred_file:
    test_prediction_mult = [float(label) 
                            for label in pred_file.readlines()]

In [262]:
accuracy_score(test_labels_mult, test_prediction_mult)

0.8473344103392568

Получили высокий показатель точности, в сравнении с прошлыми моделями.

## Neural network

In [443]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.airline_sentiment, test_size=0.1)

Проведем токенизацию

In [444]:
tk = Tokenizer(num_words=10000)
tk.fit_on_texts(X_train)

X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

Функция создает матрицу нулей, после чего записывает на соответствующие позиции значения токенизированных текстов

In [445]:
def one_hot_seq(seqs, nb_features = 10000):
    ohs = np.zeros((len(seqs), nb_features))
    for i, s in enumerate(seqs):
        ohs[i, s] = 1.0
    return ohs

X_train_oh = one_hot_seq(X_train_seq)
X_test_oh = one_hot_seq(X_test_seq)

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)

X_train_rest, X_valid, y_train_rest, y_valid = train_test_split(X_train_oh, y_train_oh, test_size=0.1)

Используем нейросеть с одним скрытым слоем, активационными функциями relu на входном и скрытом слое, softmax на выходном. Также, добавим дропаут против переобучения.

In [455]:
nn = models.Sequential()
nn.add(layers.Dense(128, init = 'uniform', activation='relu', input_shape=(10000,)))
nn.add(layers.Dropout(0.6))
nn.add(layers.Dense(128,init = 'uniform', activation='relu'))
nn.add(layers.Dropout(0.6))
nn.add(layers.Dense(3, activation='softmax'))

В качестве функции потерь будем использовать кроссэнтропию. Прогоним на 10 эпохах.

In [452]:
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
nn.fit(X_train_rest,y_train_rest, batch_size = 128, nb_epoch = 10)

Epoch 1/10
11858/11858 [==============================] - 3s 259us/step - loss: 0.4787 - acc: 0.7760
Epoch 2/10
11858/11858 [==============================] - 2s 134us/step - loss: 0.3583 - acc: 0.8374
Epoch 3/10
11858/11858 [==============================] - 2s 136us/step - loss: 0.2690 - acc: 0.8884
Epoch 4/10
11858/11858 [==============================] - 2s 134us/step - loss: 0.2124 - acc: 0.9148
Epoch 5/10
11858/11858 [==============================] - 2s 136us/step - loss: 0.1665 - acc: 0.9375
Epoch 6/10
11858/11858 [==============================] - 2s 134us/step - loss: 0.1377 - acc: 0.9475
Epoch 7/10
11858/11858 [==============================] - 2s 133us/step - loss: 0.1123 - acc: 0.9598
Epoch 8/10
11858/11858 [==============================] - 2s 133us/step - loss: 0.0905 - acc: 0.9686
Epoch 9/10
11858/11858 [==============================] - 2s 133us/step - loss: 0.0795 - acc: 0.9723
Epoch 10/10
11858/11858 [==============================] - 2s 135us/step - loss: 0.0649 - a

In [453]:
pred = nn.predict(X_valid)

Учитывая, что получаем вероятностную оценку, в качестве метрики будем использовать не accuracy, а roc_auc

In [454]:
roc_auc_score(y_valid, pred)

0.9160354078458003

# Выводы по проделанной работе

1. Загрузили датасет, провели анализ распределения целевой переменной (выявлен дисбаланс классов), отложили 10% данных для проверки качества работы.

2. Сделали oversampling выборки чтобы сбалансировать классы.

3. Привели 4 возможных препроцессинга.

4. На 8 алгоритмах классификации и 5 наборах данных (4 препроцессинга+без обработки) провели оценку качества на кросс-валидации и отложенной выборке с использованием CountVectorizer и TfidfVectorizer. Для двух лучших алгоритмов каждого векторайзера провели поиск по сетке. Accuracy порядка 0.79-0.81.

5. Word2vec показал относительно низкое качество - пайплайн с Rf показал accuracy порядка 0.64. На остальных классификаторах accuracy оказался еще ниже. Дальнейшее рассмотрение word2vec опустили. Это связано с тем, что тексты твитов весьма короткие. На больших текстах Word2vec (ну и с достаточным объемом выборки) обычно показывает лучшее качество.

6. Vowpal wabbit показал accuracy порядка 0.85. Изменение параметров алгоритма не улучшало этот показатель.

7. Нейронная сеть с одним скрытым слоем показала accuracy на отложенной выборке порядка 0.9, что является лучшим показателем из всех ранее рассмотренных моделей.

8. Как еще повысить качество моделей: стекинг алгоритмов, дополнительные данные (например, напарсить новые с использованием beautifulsoup/scrapy).